<a href="https://colab.research.google.com/github/heejinohn/lzho/blob/heejinohn/qrtopt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wrds

     |████████████████████████████████| 3.4 MB 10.2 MB/s 


In [2]:
import wrds

In [3]:
db = wrds.Connection()

Enter your WRDS username [root]:hohn
Enter your password:··········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [4]:
db.describe_table('optionm','opvold')

Approximately 61360128 rows in optionm.opvold.


,name,nullable,type
0,secid,True,DOUBLE_PRECISION
1,date,True,DATE
2,cp_flag,True,VARCHAR(1)
3,volume,True,DOUBLE_PRECISION
4,open_interest,True,DOUBLE_PRECISION


In [7]:
tqry = """
        select * from optionm.opvold
        where date >= '2017/01/01' and
           cp_flag is null        
"""

cpqry = """
        select * from optionm.opvold
        where date >= '2017/01/01' and
           cp_flag is not null
"""

tvol = db.raw_sql(tqry,date_cols='date')
cpvol = db.raw_sql(cpqry,date_cols='date')

In [8]:
tvol.secid = tvol.secid.astype(int)
cpvol.secid = cpvol.secid.astype(int)

In [9]:
tvol.rename({'volume':'tvol','open_interest':'tinterest'},
             axis=1,inplace=True)

In [10]:
list(cpvol)

['secid', 'date', 'cp_flag', 'volume', 'open_interest']

In [11]:
cpvol = cpvol.pivot_table(index=['secid','date'],columns='cp_flag',
                  values=['volume','open_interest'])

In [12]:
cpvol.columns = ['_'.join(col) for col in cpvol.columns.values]

In [13]:
tvol.drop('cp_flag',inplace=True,axis=1)

In [14]:
cpvol['int_gap'] = cpvol['open_interest_C']- cpvol['open_interest_P']

In [16]:
cpvol.reset_index(inplace=True)

In [17]:
tvol = tvol.set_index(['secid','date']).sort_index().reset_index()

In [18]:
tvol.to_stata('tvol.dta')
cpvol.to_stata('cpvol.dta')